# Working Area

## 效能優化＿實驗區

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
import concurrent.futures
from datetime import datetime

In [21]:
START = datetime.now()

def stop_watch():
    """
    A stop_watch to time a code segment 
    in order to monitor performance
    """
    global START  #<----------------------------------不加global修飾詞會出現UnboundLocalError，但加了感覺很暴力XD
    END = datetime.now()
    timeSpent = str(END - START) 

    START = END
    return timeSpent

In [22]:
START = datetime.now()

post_urls = []
for page in range(1, 3): #總共310頁 #想測試一下效能看是否能優化,因為我用了tmp處理，是否等於寫了兩次？ Ans:不會，瓶頸其實在於requests.get(URL)
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}".format(page)    
    resp = requests.get(URL)
    print('requests.get = ' + stop_watch())
    soup = BeautifulSoup(resp.text, "html5lib")
    tmp = []
    tmp += [a.get('href') for a in soup.select('tbody > tr > th > a.xst')]
    print('tmp = ' + stop_watch())
    post_urls += tmp[2:]
    print('post_urls = ' + stop_watch())
post_urls




requests.get = 0:00:02.783487
tmp = 0:00:00.205710
post_urls = 0:00:00.000184
requests.get = 0:00:01.794263
tmp = 0:00:00.172709
post_urls = 0:00:00.000141


['http://www.wetalk.tw/thread-70892-1-1.html',
 'http://www.wetalk.tw/thread-70887-1-1.html',
 'http://www.wetalk.tw/thread-70647-1-1.html',
 'http://www.wetalk.tw/thread-70829-1-1.html',
 'http://www.wetalk.tw/thread-70802-1-1.html',
 'http://www.wetalk.tw/thread-70549-1-1.html',
 'http://www.wetalk.tw/thread-70766-1-1.html',
 'http://www.wetalk.tw/thread-70689-1-1.html',
 'http://www.wetalk.tw/thread-70674-1-1.html',
 'http://www.wetalk.tw/thread-70318-1-1.html',
 'http://www.wetalk.tw/thread-70635-1-1.html',
 'http://www.wetalk.tw/thread-70584-1-1.html',
 'http://www.wetalk.tw/thread-70565-1-1.html',
 'http://www.wetalk.tw/thread-70573-1-1.html',
 'http://www.wetalk.tw/thread-70551-1-1.html',
 'http://www.wetalk.tw/thread-70520-1-1.html',
 'http://www.wetalk.tw/thread-70518-1-1.html',
 'http://www.wetalk.tw/thread-70488-1-1.html',
 'http://www.wetalk.tw/thread-70486-1-1.html',
 'http://www.wetalk.tw/thread-70471-1-1.html',
 'http://www.wetalk.tw/thread-70297-1-1.html',
 'http://www.

In [53]:
START = datetime.now()

news_list = []
count = 0

for url in post_urls:
    news = {}
    resp_c = requests.get(url)
    soup_c = BeautifulSoup(resp_c.text, 'html5lib')   
    print('requests.get與resp_c.text  = ' + stop_watch())
    
    news["title"]    = soup_c.select("h1.ts")[0].text.replace('\n','')
    news["datetime"] = soup_c.select('span.date-show-info')[0].text.replace('發表','').strip()
    news["reporter"] = soup_c.select('a[itemprop="author"]')[0].text
    news["media"]    = "wetalk"
    news["category"] = "Forum"
    news["hash"]     = hash(resp.text)
    news["url"]      = url
    news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').strip()
    news["comments"] = []
    print('all except comments = ' + stop_watch())
    
    for i in range(len(soup_c.select("td.info-post-td > a"))):
        news["comments"].append({})   
        news["comments"][i]["comment_content"] = "".join([p.text for p in soup_c.select('[id^="postmessage_"] > div[align="left"]')])
        news["comments"][i]["datetime"] = soup_c.select("span.date-show-info")[0].text.replace('\n','').replace('發表','').strip()
        news["comments"][i]["user_id"] = soup_c.select("td.info-post-td > a")[0].text
    print('comments            = ' + stop_watch())
    
    news_list.append(news)
    count+=1
    if 0 == count%2:  #林冠廷說0寫在前面比較好？原因他忘記了???
        with open("./News_Crawler/wetalk/wetalk_%s.txt"%count, 'w', encoding='UTF-8') as f:
            jd = json.dumps(news_list, ensure_ascii=False, indent=4)
            f.write(jd)
            news_list[:] = [] #聰明的作法，不然就要使用暴力的global修飾詞指派全域變數news_list
    print('write into the file = ' + stop_watch())
    #測測看用count還有length熟快熟慢～～～

#     print(news)
    

        
      

r.get與resp_c.text  = 0:00:02.269349
all except comments = 0:00:00.009149
comments            = 0:00:00.081744
write into the file = 0:00:00.000129
r.get與resp_c.text  = 0:00:01.718565
all except comments = 0:00:00.002914
comments            = 0:00:00.005718
write into the file = 0:00:00.003060
r.get與resp_c.text  = 0:00:01.507018
all except comments = 0:00:00.003052
comments            = 0:00:00.003413
write into the file = 0:00:00.000311
r.get與resp_c.text  = 0:00:01.235514
all except comments = 0:00:00.002968
comments            = 0:00:00.003389
write into the file = 0:00:00.000791
r.get與resp_c.text  = 0:00:02.082076
all except comments = 0:00:00.003273
comments            = 0:00:00.003559
write into the file = 0:00:00.000266
r.get與resp_c.text  = 0:00:02.346113
all except comments = 0:00:00.003423
comments            = 0:00:00.010516
write into the file = 0:00:00.000801
r.get與resp_c.text  = 0:00:01.409996
all except comments = 0:00:00.004374
comments            = 0:00:00.021695
write in

In [ ]:
#to cathy:
#這裏小段程式邏輯有問題xD len('tbody > tr > th > a.xst')指的是'字串'的長度喔ＸＤＤ'  

## 模組化＿實驗區

In [54]:
import requests
from bs4 import BeautifulSoup
import re
import json
import concurrent.futures
from datetime import datetime

In [ ]:
def stop_watch():
    """
    A stop_watch to time a code segment 
    in order to monitor performance
    """
    global START  #<----------------------------------不加global修飾詞會出現UnboundLocalError，但加了感覺很暴力XD
    END = datetime.now()
    timeSpent = str(END - START) 

    START = END
    return timeSpent

In [59]:
def get_url_links(page, post_urls):  
    """
    Read through HTML content and returns a list of url_links
    of each post in the category "politics" at Wetalk.tw
    """
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}".format(page)    
    resp = requests.get(URL)

    soup = BeautifulSoup(resp.text, "html5lib")
    tmp_urls = []
    tmp_urls.append([a.get('href') for a in soup.select('tbody > tr > th > a.xst')])
    post_urls += tmp_urls[2:]

In [75]:
def save_posts_info(post_urls):
    """
    Save infomations of the post to a dictionary named "news",
    then append "news" to the "news_list"
    """
    news_list = []
    count = 0

    for url in post_urls:
        news = {}
        resp_c = requests.get(url)
        soup_c = BeautifulSoup(resp_c.text, 'html5lib')   

        news["title"]    = soup_c.select("h1.ts")[0].text.replace('\n','')
        news["datetime"] = soup_c.select('span.date-show-info')[0].text.replace('發表','').strip()
        news["reporter"] = soup_c.select('a[itemprop="author"]')[0].text
        news["media"]    = "wetalk"
        news["category"] = "Forum"
        news["hash"]     = hash(resp_c.text)
        news["url"]      = url
        news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').strip()
        news["comments"] = []
        for i in range(len(soup_c.select("td.info-post-td > a"))):
            news["comments"].append({})   
            news["comments"][i]["comment_content"] = "".join([p.text for p in soup_c.select('[id^="postmessage_"] > div[align="left"]')])
            news["comments"][i]["datetime"] = soup_c.select("span.date-show-info")[0].text.replace('\n','').replace('發表','').strip()
            news["comments"][i]["user_id"] = soup_c.select("td.info-post-td > a")[0].text
        news_list.append(news)
        count += 1
        if 0 == count%2:  #這裏改成100，每100個post輸出一次
            output_as_json(count, 'wetalk', news_list)
            news_list[:] = []
            print('輸出一個檔案囉～')

def output_as_json(index, media, x):  
    """
    Write data in json format into .txt files
    """
    with open("./News_Crawler/{}/{}_{}.txt".format(media, media, index), 'w', encoding='UTF-8') as f:
        jd = json.dumps(x, ensure_ascii=False, indent=4)
        f.write(jd)

        
      

In [76]:
def main():
    urls = get_url_links()
    save_posts_info(urls)
    
    

In [77]:
if __name__ == "__main__":        
    START = datetime.now()
    main()
    print('main = ' + stop_watch()) 

輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
main = 0:00:42.852742


In [ ]:
#還沒加上class

## 多執行緒＿實驗區 

In [159]:
import requests
from bs4 import BeautifulSoup
import re
import json
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime

In [160]:
def stop_watch():
    """
    A stop_watch to time a code segment 
    in order to monitor performance
    """
    global START  #<----------------------------------不加global修飾詞會出現UnboundLocalError，但加了感覺很暴力XD
    END = datetime.now()
    timeSpent = str(END - START) 

    START = END
    return timeSpent

In [161]:
def get_url_links(page, post_urls):  
    """
    Read through HTML content and returns a list of url_links
    of each post in the category "politics" at Wetalk.tw
    """
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}".format(page)    
    resp = requests.get(URL)

    soup = BeautifulSoup(resp.text, "html5lib")
    tmp_urls = []
    tmp_urls.extend([a.get('href') for a in soup.select('tbody > tr > th > a.xst')])
    post_urls += tmp_urls[2:]

In [165]:
def save_posts_info(url, news_list, count):
    """
    Save infomations of the post to a dictionary named "news",
    then append "news" to the "news_list"
    and output as json in a .txt file every 2 posts
    """
    news = {}
    resp_c = requests.get(url)
    soup_c = BeautifulSoup(resp_c.text, 'html5lib')   

    news["title"]    = soup_c.select("h1.ts")[0].text.replace('\n','')
    news["datetime"] = soup_c.select('span.date-show-info')[0].text.replace('發表','').strip()
    news["reporter"] = soup_c.select('a[itemprop="author"]')[0].text
    news["media"]    = "wetalk"
    news["category"] = "Forum"
    news["hash"]     = hash(resp_c.text)
    news["url"]      = url
    news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').strip()
    news["comments"] = []
    for i in range(len(soup_c.select("td.info-post-td > a"))):
        news["comments"].append({})   
        news["comments"][i]["comment_content"] = "".join([p.text for p in soup_c.select('[id^="postmessage_"] > div[align="left"]')])
        news["comments"][i]["datetime"] = soup_c.select("span.date-show-info")[0].text.replace('\n','').replace('發表','').strip()
        news["comments"][i]["user_id"] = soup_c.select("td.info-post-td > a")[0].text
    news_list.append(news)
    count.append('SUCCESS')  #這裏我使用list做為計數器，因為list傳的是記憶體位址，各個thread執行完去append都不會出現問題
    if 0 == len(count)%2:  #這裏改成100，每100個post輸出一次
        output_as_json(len(count), 'wetalk', news_list)
        news_list[:] = []
        print('輸出一個檔案囉～')

def output_as_json(index, media, x):  
    """
    Write data in json format into .txt files
    """
    with open("./News_Crawler/{}/{}_{}.txt".format(media, media, index), 'w', encoding='UTF-8') as f:
        jd = json.dumps(x, ensure_ascii=False, indent=4)
        f.write(jd)

        
      

In [166]:
def main(THREAD_NUM):
    threads = ThreadPoolExecutor(THREAD_NUM)
    
    post_urls = []
    futures = [threads.submit(get_url_links, page, post_urls) for page in range(1, 2)]
    wait(futures)
        
    news_list = []
    count = []
    futures = [threads.submit(save_posts_info, post_url, news_list, count) for post_url in post_urls]
    wait(futures)

    

In [167]:
if __name__ == "__main__":        
    START = datetime.now()
    main(10)
    print('main = ' + stop_watch()) 

輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
main = 0:00:19.215276


## Load into MongoDB_實驗區

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime
from pymongo import MongoClient

In [2]:
def stop_watch():
    """
    A stop_watch to time a code segment 
    in order to monitor performance
    """
    global START  #<----------------------------------不加global修飾詞會出現UnboundLocalError，但加了感覺很暴力XD
    END = datetime.now()
    timeSpent = str(END - START) 

    START = END
    return timeSpent

In [3]:
def get_url_links(page, post_urls):  
    """
    Read through HTML content and returns a list of url_links
    of each post in the category "politics" at Wetalk.tw
    """
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}".format(page)    
    resp = requests.get(URL)

    soup = BeautifulSoup(resp.text, "html5lib")
    tmp_urls = []
    tmp_urls.extend([a.get('href') for a in soup.select('tbody > tr > th > a.xst')])
    post_urls += tmp_urls[2:]

In [4]:
def save_posts_info(url, news_list, count):
    """
    Save infomations of the post to a dictionary named "news",
    then append "news" to the "news_list",
    load the "news" record into MongoDB,
    and finally output as json in a .txt file every 2 posts
    """
    news = {}
    resp_c = requests.get(url)
    soup_c = BeautifulSoup(resp_c.text, 'html5lib') 

    news["title"]    = soup_c.select("h1.ts")[0].text.replace('\n','')
    news["datetime"] = soup_c.select('span.date-show-info')[0].text.replace('發表','').strip()
    news["reporter"] = soup_c.select('a[itemprop="author"]')[0].text
    news["media"]    = "wetalk"
    news["category"] = "Forum"
    news["hash"]     = hash(resp_c.text)
    news["url"]      = url
    news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').strip()
    news["comments"] = []
    
    for i in range(len(soup_c.select("td.info-post-td > a"))):
        news["comments"].append({})   
        news["comments"][i]["comment_content"] = "".join([p.text for p in soup_c.select('[id^="postmessage_"] > div[align="left"]')])
        news["comments"][i]["datetime"] = soup_c.select("span.date-show-info")[0].text.replace('\n','').replace('發表','').strip()
        news["comments"][i]["user_id"] = soup_c.select("td.info-post-td > a")[0].text
    news_list.append(news)
    count.append('SUCCESS')  #這裏我使用list做為計數器，因為list傳的是記憶體位址，各個thread執行完去append都不會出現問題
    load_into_mongodb('testDatabase', 'testCollection', news)

    if 0 == len(count)%2:  #這裏改成100，每100個post輸出一次
        output_as_json(len(count), 'wetalk', news_list)
        news_list[:] = []
        print('輸出一個檔案囉～')

def output_as_json(index, media, x):    
    """
    Write data in json format into .txt files
    """
    with open("./News_Crawler/{}/{}_{}.txt".format(media, media, index), 'w', encoding='UTF-8') as f:
        jd = json.dumps(x, ensure_ascii=False, indent=4)
        f.write(jd)
# 這裏需要注意的是資料夾要自己先建好，with open("...",'w')只會幫你自動產生檔案，而不會產生資料夾
# JSON encoder and decoder的介紹
# https://docs.python.org/3/library/json.html
# 了解JSON格式
# http://j796160836.pixnet.net/blog/post/30530326-瞭解json格式

        
      

In [5]:
# def load_into_mongodb(db_name, collection_name, doc):
#     """
#     Loads data into MongoDB
#     """
#     db = client[db_name]
#     collection = db[collection_name]
#     collection.insert_one(doc)
    
# # 之後應改寫判斷條件，然後如果單筆document使用.insert_one()，多筆則使用.insert_many()
# # 注意這裡要使用pymongo的語法，而非mongoscript，否則無法使用“變數” eg. db.collection_name.insert(record)建立的db會叫做collection_name而非tesCollection
# # 詳見：
# # 如何利用pymongo建立connection, db, collection: http://api.mongodb.com/python/current/tutorial.html
# # 如何利用pymongo進行Collection level operations, eg. insert_many(): http://api.mongodb.com/python/current/api/pymongo/collection.html

# # bulk_write(Requests: Requests are passed as a list of write operation instances ( InsertOne, UpdateOne, UpdateMany, ReplaceOne, DeleteOne, or DeleteMany).), insert_one(document: The document to insert.), insert_many(documents: A iterable of documents to insert.)
# # http://api.mongodb.com/python/current/api/pymongo/collection.html



In [6]:
def load_into_mongodb(db_name, collection_name, doc):
    """
    Loads data into MongoDB
    """
    db = client[db_name]
    collection = db[collection_name]
    collection.insert(doc)
    
# 之後應改寫判斷條件，然後如果單筆document使用.insert_one()，多筆則使用.insert_many()
# 注意這裡要使用pymongo的語法，而非mongoscript，否則無法使用“變數” eg. db.collection_name.insert(record)建立的db會叫做collection_name而非tesCollection
# 詳見：
# 如何利用pymongo建立connection, db, collection: http://api.mongodb.com/python/current/tutorial.html
# 如何利用pymongo進行Collection level operations, eg. insert_many(): http://api.mongodb.com/python/current/api/pymongo/collection.html

# bulk_write(Requests: Requests are passed as a list of write operation instances ( InsertOne, UpdateOne, UpdateMany, ReplaceOne, DeleteOne, or DeleteMany).), insert_one(document: The document to insert.), insert_many(documents: A iterable of documents to insert.)
# http://api.mongodb.com/python/current/api/pymongo/collection.html




In [7]:
def show_db_data(db_name, collection_name):
    """
    Query data from MongoDB,
    and print it to the console
    """
    db = client[db_name]
   # utilize for-loop to iterate through curser-object
    for doc in db.collection_name.find():
        print(doc) 

In [8]:
def main(THREAD_NUM):
    threads = ThreadPoolExecutor(THREAD_NUM)
    
    post_urls = []
    futures = [threads.submit(get_url_links, page, post_urls) for page in range(1, 2)]
    wait(futures)
        
    news_list = []
    count = []
    futures = [threads.submit(save_posts_info, post_url, news_list, count) for post_url in post_urls]
    wait(futures)
    
    #show_db_data('testDatabase', 'testCollection')

    

In [10]:
if __name__ == "__main__":        
    START = datetime.now()
    client = MongoClient('localhost', 27017, maxPoolSize=None) #這裡需要指名你要連線的MongoDB所在的ip和port
    main(10)
    print('main = ' + stop_watch()) 

/Users/natashapan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


main = 0:00:22.412879


## Jieba

## Final Version

In [ ]:
Class WetalkCrawler():
    def __init__(self):
        pass
    def __

In [110]:
class Test():
    def __init__(self, name):
        self.name = name
        
    def add(self, x, y):
        return x+y, self.name, x, y

In [ ]:
from pymongo import MongoClient
import json
client = MongoClient('localhost',27017)
#db_name = testDatabase
#collection_name = testCollection
db = client['testDatabase']
# db.testCollection.insert({"key":"value"})

with open('D:/woodnata_note/test_data') as f:
    str_cont = f.read()
    list_cont = json.loads(str_cont)
    # insert a 'list' of data using method 'insert_many()'
    # [dbname].[collection].method
    db.testCollection.insert_many(list_cont)

#we use for loop to iterate the real data from curser
for doc in db.testCollection.find():
    print(doc)


## Improving performance and profiling__實驗區

* 7 tips to Time Python scripts and control Memory & CPU usage
    <br> http://www.marinamele.com/7-tips-to-time-python-scripts-and-control-memory-and-cpu-usage


* google: measure performance bottleneck python


* Performance and Profiling
Q: What is Software Profiling?
A: The act of using instrumentation to objectively measure the performance of your application. 
“Performance” can be a measure of any of the following:
    1. resource use (CPU, memory)
    2. frequency or duration of function calls
    3. wall clock execution time of part or all of your application
    <br>http://uwpce-pythoncert.github.io/SystemDevelopment/profiling.html


* stopwatch.py
    <br>https://gist.github.com/igniteflow/1253276


* timeit.timeit()
    <br> https://docs.python.org/3.6/library/timeit.html
